## **Instalação dos Pacotes**

In [ ]:
# !pip install -U -q langchain langchain-core langchain-openai langchain-community pydantic

## **Configuração das Variáveis de Ambiente**

Configuração da chave de API da OpenAI para autenticação

In [1]:
import os
from pathlib import Path


def load_env() -> None:
    """Load environment variables from the appropriate source.
    
    If running in Google Colab, loads secrets from Colab's userdata.
    Otherwise, loads variables from a local .env file using python-dotenv.
    """
    try:
        import google.colab
        from google.colab import userdata

        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY")

    except ImportError:
        from dotenv import load_dotenv
        load_dotenv()


load_env()

## **Imports dos Pacotes Usados no Notebook**

Importação dos componentes principais do LangChain para construção de aplicações com LLMs

In [126]:
import tqdm

In [2]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## **Inicialização do Modelo LLM**

Criação da instância do modelo ChatGPT que será utilizado em todos os exemplos

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4.1-nano")

print("Modelo LLM inicializado com sucesso!")

Modelo LLM inicializado com sucesso!


In [4]:
llm.invoke("olá, tudo bem?")

AIMessage(content='Olá! Tudo ótimo, obrigado. Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_336de151a4', 'id': 'chatcmpl-DDiMhPdHpHfFhhMZlUDI0Se9T1apL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9d09-85b1-7fe1-b486-8ec3427a79a9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
file_url = "https://raw.githubusercontent.com/renansantosmendes/synapseai_data/refs/heads/main/company_context.txt"

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(file_url, encoding="utf-8")
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [67]:
from pydantic import BaseModel, Field, constr
from typing import List, Dict

class ProductAnalysis(BaseModel):
    """Associação de um produto aos seus problemas."""
    product_name: str = Field(description="Nome do produto")
    problems: List[str] = Field(description="Lista de problemas identificados para este produto")

class ProductsProblems(BaseModel):
    """Estrutura final para a extração de múltiplos produtos."""
    content: List[ProductAnalysis] = Field(description="Lista de produtos e seus problemas")

In [8]:
llm_structured = llm.with_structured_output(ProductsProblems)

In [ ]:
products_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere a descrição do
produto e seu nome. O nome deve ser o mais curto possível, sem adicionar a descrição.
O campo descrição deve ser feito de forma separada do nome do produto
</context>

<company_description>
{company_description}
</company_description>
""")

In [118]:
products_improvement_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere a descrição do
produto e seu nome. O nome deve ser o mais curto possível, sem adicionar a descrição.
O campo descrição deve ser feito de forma separada do nome do produto

Siga os passos para criar toda parte detalhada:
- Crie os requisitos funcionais e não funcionais que o produto deve apresentar
- Use o contexto do produto para criar ainda mais detalhes sobre os seguintes pontos: ferramentas usadas, 
modelos usados, abordagem, integrações com outros sistemas, integrações e ferramentas de observabilidade, 
detalhes do deploy, fonte de dados, privacidade, detalhes da escalabilidade (volume, cargas, uso de memória), 
versionamento de prompts e modelos

Garanta que a linguagem é uma linguagem técnica, evite a todo custo uma linguagem de négocios.
</context>

<product_name>
{product_name}
</product_name>

<product_description>
{product_description}
</product_description>
""")

In [119]:
class ProductEntry(BaseModel):
    """Representa um produto individual da companhia e sua definição oficial."""
    name: str = Field(
        description="O nome oficial do produto (ex: 'Cartão Platinum', 'App Invest', 'CDB Fácil')"
    )
    description: str = Field(
        description="A descrição detalhada e técnica de todas das funcionalidades e regras do produto encontrada no texto"
    )
    
class ProductSpecification(BaseModel):
    product_name: str = Field(description="Nome do produto")
    detailed_description: str = Field(description="Descrição técnica detalhada nos mínimos detalhesdo produto")

class CompanyProductCatalog(BaseModel):
    """Catálogo completo de produtos extraído dos documentos da empresa."""
    products: List[ProductEntry] = Field(
        description="Lista de todos os produtos identificados no texto contendo suas descrições detalhadas"
        )

In [120]:
product_chain = (
    products_prompt | 
    llm.with_structured_output(
        schema=CompanyProductCatalog, 
        method="function_calling"
        )
    )

product_improvement_chain = (
    products_improvement_prompt | 
    init_chat_model(
        model="openai:gpt-4.1-mini"
        ).with_structured_output(
            schema=ProductSpecification, 
            method="function_calling"
        )
)

## Criação da lista de produtos + descrições

In [112]:
catalog = product_chain.invoke(
    {
        "company_description": docs[0].page_content
    }
)

In [113]:
catalog.products

[ProductEntry(name='FinBrain', description='O FinBrain é um copiloto financeiro corporativo que transforma dados financeiros complexos em insights claros, explicáveis e acionáveis. Ele permite que executivos, controllers e analistas façam perguntas em linguagem natural sobre resultados financeiros, projeções e indicadores-chave, reduzindo drasticamente o tempo gasto em análises manuais e relatórios. Com o FinBrain, decisões financeiras tornam-se mais rápidas, fundamentadas e transparentes, melhorando a comunicação entre áreas técnicas e a alta liderança.'),
 ProductEntry(name='RiskGen', description='O RiskGen é uma plataforma de IA generativa que automatiza e padroniza análises de risco regulatório, compliance e auditoria, permitindo que organizações lidem com contratos e normas complexas com muito mais eficiência e segurança. Ele reduz riscos operacionais, aumenta a consistência das análises e fornece suporte técnico confiável para áreas jurídicas, de compliance e de auditoria interna

In [114]:
catalog_dict = {item['name']: item['description'] for item in catalog.model_dump()["products"]}

In [115]:
current_product = "FinBrain"
print(catalog_dict.get(current_product))

O FinBrain é um copiloto financeiro corporativo que transforma dados financeiros complexos em insights claros, explicáveis e acionáveis. Ele permite que executivos, controllers e analistas façam perguntas em linguagem natural sobre resultados financeiros, projeções e indicadores-chave, reduzindo drasticamente o tempo gasto em análises manuais e relatórios. Com o FinBrain, decisões financeiras tornam-se mais rápidas, fundamentadas e transparentes, melhorando a comunicação entre áreas técnicas e a alta liderança.


## Adição de detalhes ao produto

In [121]:
detailed_product = product_improvement_chain.invoke(
    {
        "product_name": current_product,
        "product_description": catalog_dict.get(current_product, "")
    }
)

In [122]:
detailed_product.detailed_description

'Requisitos Funcionais:\n1. Interface de Pergunta em Linguagem Natural: Permite que usuários façam perguntas sobre dados financeiros e receban respostas precisas e explicáveis.\n2. Análise de Dados Financeiros: Realiza processamento e análise de grandes volumes de dados financeiros corporativos, como balanços, demonstrativos de resultados, e indicadores-chave.\n3. Geração de Insights e Projeções: Oferece projeções financeiras baseadas em dados históricos e modelos preditivos.\n4. Explicabilidade dos Resultados: Fornece a fundamentação lógica e técnica para os insights gerados para suportar decisões.\n5. Suporte a Múltiplos Usuários Corporativos: Permite o uso por executivos, controllers e analistas com diferentes níveis de permissão.\n6. Dashboards Interativos: Visualização de dados e insights por meio de dashboards interativos e customizáveis.\n\nRequisitos Não Funcionais:\n1. Segurança e Privacidade: Garantia de confidencialidade dos dados financeiros, com criptografia ponta a ponta 

In [133]:
detailed_catalog_dict = {}
customer_dict = {}

pbar = tqdm.tqdm(catalog_dict.keys())
for product_key in pbar:
    pbar.set_description(f"Processing: {product_key}")
    
    detailed_product = product_improvement_chain.invoke(
        {
            "product_name": product_key,
            "product_description": catalog_dict.get(product_key, "")
        }
    )
    
    detailed_catalog_dict[product_key] = detailed_product.detailed_description
    
    

print("products description improved!")

Processing: ClinicaGPT: 100%|██████████| 5/5 [00:43<00:00,  8.78s/it]      

products description improved!


In [135]:
current_product

'FinBrain'

In [192]:
detailed_catalog_dict

{'FinBrain': 'O FinBrain é um copiloto financeiro corporativo avançado que utiliza processamento de linguagem natural (PLN) e aprendizado automático para transformar dados financeiros complexos em insights claros, explicáveis e acionáveis. Funcionalmente, o produto deve suportar consulta em linguagem natural sobre dados financeiros históricos, projeções e indicadores-chave de desempenho (KPIs), fornecendo respostas precisas e contextualizadas. Além disso, deve permitir integração com sistemas ERP, CRM e bancos de dados financeiros para extração automática e atualização em tempo real dos dados. Não funcionalmente, o produto deve garantir alta disponibilidade (>99,9%), baixa latência de respostas (<200ms), segurança robusta com criptografia ponta a ponta e conformidade com normas de privacidade de dados (ex: LGPD). A arquitetura do FinBrain utiliza modelos de linguagem grandes (LLMs), fine-tuned para domínio financeiro, associados a modelos estatísticos de séries temporais para previsão 

## Criação do público de cada produto

In [136]:
class CustomerAnalysis(BaseModel):
    """Análise detalhada de um cliente potencial para um produto."""
    customer_name: str = Field(description="Nome do cliente")
    customer_description: str = Field(description="Descrição detalhada do cliente")

class PublicTargetAnalysis(BaseModel):
    """Análise detalhada do público-alvo de um produto, incluindo jornada do cliente e exemplos reais."""
    target_audience_description: str = Field(description="Descrição detalhada do público-alvo para o produto")
    customer_journey: str = Field(description="Descrição da jornada de uso do produto, desde o primeiro contato até o uso recorrente")
    example_customers: List[CustomerAnalysis] = Field(
        description="Lista de possíveis clientes reais que se encaixariam no público-alvo, com breve descrição de cada um"
    )

In [137]:
customer_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere uma descrição
de um público alvo para o produto, uma possivel jornada de uso do produto, 
desde o primeiro contato do cliente com o produto, até o uso recorrente do mesmo.
Além disso, gere uma lista de possíveis clientes que sejam reais e que se encaixariam 
no público alvo do produto com uma breve descrição de cada um desses clientes.
</context>

<company_description>
{company_description}
</company_description>

Considere o produto "{product_name}" e sua descrição "{product_description}" para conseguir gerar problemas mais
contextualizados
""")

In [138]:
customer_structured_llm = llm.with_structured_output(PublicTargetAnalysis, method="function_calling")
customer_chain = customer_prompt | customer_structured_llm

In [ ]:
customers_dict = {}
for product_key in detailed_catalog_dict.keys():
    customers = customer_chain.invoke(
        {
            "company_description": docs[0].page_content,
            "product_name": product_key,
            "product_description": catalog_dict.get(product_key)
        }
    )
    customers_dict[product_key] = customers

In [140]:
customers.model_dump().get("example_customers")

[{'customer_name': 'João Silva',
  'customer_description': 'Controller de uma grande empresa do setor de energia, responsável por consolidar relatórios financeiros trimestrais e estratégicos, buscando agilidade para análises complexas e conformidade regulatória.'},
 {'customer_name': 'Maria Fernandes',
  'customer_description': 'Diretora financeira de uma holding de investimentos, que necessita de insights rápidos sobre a saúde financeira de diferentes portfólios e de suporte na tomada de decisões de alocação de recursos.'},
 {'customer_name': 'Carlos Pereira',
  'customer_description': 'Analista de finanças de uma fintech com alto volume de transações e necessidade de relatórios precisos para investidores e órgãos reguladores, buscando automação e confiabilidade nos dados.'},
 {'customer_name': 'Luana Costa',
  'customer_description': 'Gerente de controladoria de um conglomerado industrial, interessada em reduzir o tempo de elaboração de relatórios financeiros e melhorar a comunicação

# usar esse for aqui

In [ ]:
for current_product in catalog_dict.keys():
    print(current_product)

## Criação dos problemas para cada produto

In [141]:
problems_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes de uma empresa gere uma lista de
possíveis problemas que cada produto da empresa pode apresentar do ponto de
vista do usuário final, como por exemplo API instável, sistema demora muito
para responder,
</context>

<company_description>
{company_description}
</company_description>

Use o contexto do produto "{product_name}" e sua descrição "{product_description}" para conseguir criar cenários de problemas reais para 
a abertura de tickets de suporte contextualizados. Garanta que os problemas listados sejam realmente relacionados ao produto e não sejam 
problemas genéricos de mercado.

Use as informações a seguir para conseguir gerar os problemas de cada produto:

## Informações sobre o produto:
- Descrição do público-alvo: {target_audience_description}
- Jornada do cliente: {customer_journey}
- Exemplos de clientes: {example_customers}

Crie problemas especificos para o produto em questão. 

Gere dados que evidencie o problema para cada um dos problemas listados, como por exemplo: porcentagem de clientes afetados, 
impacto financeiro, quantidade de vezes que o problema aconteceu, horários nos quais os problemas ocorrem, etc. 

Use termos técnicos e específicos do domínio do produto para descrever os problemas, evitando generalizações e garantindo que cada problema seja claramente relacionado ao produto em questão.
""")

In [147]:
problems_chain = (
    problems_prompt | 
    llm.with_structured_output(
        schema=ProductAnalysis, 
        method="function_calling"
        )
)

In [148]:
problems_response = problems_chain.invoke({
    "product_name" : current_product,
    "product_description": detailed_catalog_dict.get(current_product),
    "company_description": docs[0].page_content,
    "target_audience_description": customers.model_dump().get("target_audience_description"),
    "customer_journey": customers.model_dump().get("customer_journey"),
    "example_customers": customers.model_dump().get("example_customers"),
})

In [159]:
product_dict = problems_response.model_dump()

In [161]:
# product_problems_dict = {item['product_name']: item['problems'] for item in problems_response.model_dump().get("problems")}
product_problems_dict = {product_dict['product_name']: product_dict['problems']}

In [162]:
product_problems_dict

{'FinBrain': ['Latência de respostas superior a 200ms durante consultas complexas de grandes volumes de dados financeiros, afetando aproximadamente 15% dos usuários em horários de pico, o que resulta em aumento de 30% no tempo de análise e dificuldades na tomada de decisões estratégicas.',
  'Inconsistência nos dados retornados em consultas de projeções financeiras de múltiplos períodos, especialmente ao trabalhar com bases de dados específicas de clientes como ERP SAP e Oracle, gerando, em média, 5 ocorrências semanais por cliente, impactando a confiabilidade dos relatórios em 25%.',
  'Falha na integração automática de dados em tempo real com sistemas ERP em 40% das tentativas de sincronização, principalmente em ambientes híbridos de nuvem privada e on-premise, levando à utilização de dados desatualizados em até 4 horas, afetando decisões de fluxo de caixa e alocação de orçamento.',
  'Problemas de indisponibilidade do serviço de API, apresentando queda de aproximadamente 2 horas por

In [163]:
import random

def get_random_element(input_list):
    """Selects random elements from a list"""
    return random.choice(input_list) if input_list else None

Produto escolhido: Seguro Vida


In [189]:
product_problems_dict.keys()

dict_keys(['FinBrain'])

In [165]:
get_random_element(product_problems_dict.get(current_product))

'Falha na integração automática de dados em tempo real com sistemas ERP em 40% das tentativas de sincronização, principalmente em ambientes híbridos de nuvem privada e on-premise, levando à utilização de dados desatualizados em até 4 horas, afetando decisões de fluxo de caixa e alocação de orçamento.'

## Execução do enriquecimento inicial, antes da criação do ticket

In [236]:
detailed_catalog_dict = {}
customers_dict = {}
product_problems_dict = {}

pbar = tqdm.tqdm(catalog_dict.keys())
for product_key in pbar:
    pbar.set_description(f"Processing: {product_key}")
    
    detailed_product = product_improvement_chain.invoke(
        {
            "product_name": product_key,
            "product_description": catalog_dict.get(product_key, "")
        }
    )
    
    detailed_catalog_dict[product_key] = detailed_product.detailed_description
    print("Product details improved!")
    
    customers = customer_chain.invoke(
        {
            "company_description": docs[0].page_content,
            "product_name": product_key,
            "product_description": detailed_catalog_dict.get(product_key)
        }
    )
    customers_dict[product_key] = customers
    print("Customer details improved!")
    
    problems_response = problems_chain.invoke({
        "product_name" : product_key,
        "product_description": detailed_catalog_dict.get(product_key),
        "company_description": docs[0].page_content,
        "target_audience_description": customers_dict[product_key].model_dump().get("target_audience_description"),
        "customer_journey": customers_dict[product_key].model_dump().get("customer_journey"),
        "example_customers": customers_dict[product_key].model_dump().get("example_customers"),
    })
    
    product_problems_dict[product_key] = problems_response.model_dump().get("problems")
    print("Product problems identified!")

print("products description improved!")

Processing: FinBrain:   0%|          | 0/5 [00:00<?, ?it/s]

Product details improved!
Customer details improved!


Processing: RiskGen:  20%|██        | 1/5 [00:18<01:13, 18.34s/it] 

Product problems identified!
Product details improved!
Customer details improved!


Processing: EduMentor AI:  40%|████      | 2/5 [00:33<00:49, 16.52s/it]

...

Product problems identified!
Product details improved!
Customer details improved!


Processing: ClinicaGPT:  80%|████████  | 4/5 [01:04<00:15, 15.64s/it]      

Product problems identified!
Product details improved!
Customer details improved!


Processing: ClinicaGPT: 100%|██████████| 5/5 [01:18<00:00, 15.74s/it]

Product problems identified!
products description improved!


## Criação dos Tickets

In [188]:
import pandas as pd
import json
from enum import Enum
from typing import List
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


class CriticalityLevel(str, Enum):
    LOW = "Baixa"
    MEDIUM = "Média"
    HIGH = "Alta"
    CRITICAL = "Crítica"

class SupportTicket(BaseModel):
    """Estrutura de um ticket de suporte com dados realistas."""
    customer_name: str = Field(description="Nome completo real (ex: João Carlos de Alencar, Maria Aparecida Souza)")
    customer_email: str = Field(description="E-mail variado (NÃO use @example.com. Use @gmail, @uol, @empresa.com.br, etc)")
    product_name: str = Field(description="Nome do produto do mercado financeiro ou tecnologia")
    issue_description: str = Field(description="Relato do problema em linguagem natural humana")
    opening_date: str = Field(description="Data de abertura (formato ISO YYYY-MM-DD)")
    criticality: CriticalityLevel = Field(description="Nível de urgência")

class TicketBatch(BaseModel):
    """Lote de tickets para exportação."""
    tickets: List[SupportTicket]

In [226]:

ticket_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um gerador de dados sintéticos de alta fidelidade para o mercado brasileiro.

    DIRETRIZES PARA REALISMO:
    - NOMES: Use nomes brasileiros reais e diversos (sobrenomes como Silva, Oliveira, Cavalcanti, Schmidt, Nogueira).
    - E-MAILS: Use domínios como @hotmail.com, @uol.com.br, @terra.com.br, @gmail.com ou domínios de empresas fictícias (@startup.io, @banco.com.br).
    Se for criar um nome de empresa fictícia, garanta que o domínio do e-mail seja condizente com o nome da empresa (ex: nome da empresa: "TechNova", e-mail: "funcionario@technova.io").
    - PROIBIÇÃO: Está proibido o uso do domínio '@example.com'.
    - DATAS: Gere datas aleatórias no ano de 2025.
    - PERSONAS: Varie entre clientes corporativos sérios e usuários comuns informais."""),
    ("human", "Gere um ticket de suporte para o produto '{product_name}' com a seguinte descrição '{product_description}' sobre o problema: '{product_problem}'."),
    ("human", "Garanta que o problema seja único e especifico para aquele cliente em questão."),
    ("human", "Gere os detalhes e dados que evidencie o problema para cada um dos problemas listados, como por exemplo: porcentagem de clientes afetados, impacto financeiro, quantidade de vezes que o problema aconteceu, horários nos quais os problemas ocorrem, etc. Use termos técnicos e específicos do domínio do produto para descrever os problemas, evitando generalizações e garantindo que cada problema seja claramente relacionado ao produto em questão.")
])

In [227]:
tickets_chain = (
    ticket_prompt | init_chat_model(
    model="openai:gpt-4.1-mini",
    temperature=0.8
    ).with_structured_output(
        schema=SupportTicket,
        method="function_calling"
        )
)

In [ ]:
import json

def salvar_incrementalmente(dados, arquivo):
    """Adiciona um dicionário ao arquivo JSON (um por linha)"""
    with open(arquivo, "a", encoding="utf-8") as f:
        json.dump(dados, f, ensure_ascii=False)
        f.write("\n")

In [271]:
number_of_tickets = 1127
tickets = []
for i in range(number_of_tickets):
    random_product = get_random_element(list(product_problems_dict.keys()))
    random_problem = get_random_element(product_problems_dict.get(random_product))
    try:
        ticket_response = tickets_chain.invoke(
            {
                "product_name": random_product,
                "product_description": detailed_catalog_dict.get(random_product, ""),
                "product_problem": random_problem
            }
        )
        
        tickets.append(ticket_response.model_dump(mode='json'))
    except Exception as e:
        print(f"Error generating ticket {i+1}: {e}")
    
    
    print(f"Ticket {i+1}:\n{ticket_response.model_dump(mode='json')}\n")
    
salvar_incrementalmente(tickets, "tickets.jsonl")

Ticket 1:
{'customer_name': 'Lucas Oliveira Schmidt', 'customer_email': 'lucas.schmidt@saudeprev.com.br', 'product_name': 'ClinicaGPT', 'issue_description': 'O cliente Lucas Oliveira Schmidt, da empresa SaúdePrev, identificou latência de resposta superior a 500ms especificamente em ambientes de alta carga, o que impacta negativamente consultas emergenciais e a elaboração de documentos clínicos rápidos. Dados coletados indicam que este problema afeta aproximadamente 12% dos usuários simultâneos na plataforma, ocorrendo principalmente entre 18h e 22h, horários de pico na área hospitalar. Foram registradas 87 ocorrências nas últimas duas semanas, com picos de latência atingindo até 850ms. Essa degradação na performance resulta em atrasos críticos na geração automática de documentos como notas de evolução e prescrições médicas, aumentando o tempo médio de consulta em até 25%, o que pode comprometer decisões clínicas ágeis. O impacto financeiro estimado para SaúdePrev é de R$45.000 mensais 

In [265]:
# sourcery skip: for-append-to-extend
dados_carregados = []
with open("tickets.jsonl", "r", encoding="utf-8") as f:
    for linha in f:
        if linha.strip():
            dados_carregados.append(json.loads(linha))